<h1> Datto RMM - Reports Summary </h1>

In [ ]:
# general purpose
import time
import datetime as dt

# Web Bot
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Data Ingest and Export
import pandas as pd
import re

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
from requests.structures import CaseInsensitiveDict

# Create Bot Object

## Create Firefox Driver

## Create Chrome Driver

In [ ]:
chrome_driver = 'D:\data_science_utilities\web_drivers\chrome\chromedriver_win32\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless')
c_driver = webdriver.Chrome(executable_path=chrome_driver)

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers[
    'User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
c_driver.get(auth_url)

In [ ]:
login_txtbox = c_driver.find_element_by_xpath("//input[@id='form_username']")
login_txtbox.send_keys((config['dattormm']['username']), Keys.ENTER)

In [ ]:
pass_txtbox = c_driver.find_element_by_xpath("//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']), Keys.ENTER)

In [ ]:
time.sleep(3)

In [ ]:
mfa_txtbox = c_driver.find_element_by_id('authy_token')
mfa_txtbox.send_keys('571023', Keys.ENTER)  # number entered manually for now /TODO: automate MFA

# Get Active Reports Summary

## Click Events to Reach Page

In [ ]:
expand_analytics_xpath = "/html/body/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[8]/div"

In [ ]:
expand_analytics = c_driver.find_element_by_xpath(expand_analytics_xpath)

In [ ]:
expand_analytics.click()

In [ ]:
reports_xpath = "/html/body/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[8]/ul/li[1]/a"

In [ ]:
reports_page_button = c_driver.find_element_by_xpath(reports_xpath)

In [ ]:
reports_page_button.click()

time.sleep(5)

## Change Page Display Count

In [ ]:
def change_page_count(web_driver, arg_page_count=100):
    time.sleep(3)

    import re

    dropdown_open = False

    try:
        web_driver.find_element_by_xpath(
            """//div[contains(@class,"ant-select-open")][contains(@class,"ant-pagination-options-size-changer")]""")
        print(f"The dropdown is expanded!")
        dropdown_open = True
    except:
        pass

    def get_dropbox_xpath():
        current_page_count_xpath = r"""//span[@class="ant-select-selection-item"][contains(@title,' / page')]"""
        current_page_count_text = c_driver.find_element_by_xpath(current_page_count_xpath).text
        current_page_count_no = re.search(r"(\d+)\s\/\s\bpage\b", current_page_count_text).group(1)
        print(f"text: {current_page_count_text}")
        print(f"Current page count: {current_page_count_no}")

        return current_page_count_xpath

    def get_selection_xpath(page_count=arg_page_count):
        print(f"Page count: {page_count}")
        selection_xpath = f"""//div[@class="ant-select-item-option-content"][contains(text(),"{page_count} / page")]"""

        return selection_xpath

    if not dropdown_open:
        count_dropbox_xpath = get_dropbox_xpath()
        web_driver.find_element_by_xpath(count_dropbox_xpath).click()

    time.sleep(1)

    count_selection_xpath = get_selection_xpath()
    web_driver.find_element_by_xpath(count_selection_xpath).click()

In [ ]:
change_page_count(c_driver, 100)

## Gather Headings and TD Text

In [ ]:
class Get_Reports():

    def __init__(self, __webdriver):
        self.__webdriver = __webdriver

    def get_reports_dataframe(self):

        df_reports = self.__get_report_body__()
        iter_page = self.__iterate_pages__()['last_page']

        while iter_page == 'false':
            time.sleep(5)
            df_current_page = self.__get_report_body__()
            df_reports = pd.concat([df_current_page, df_reports], ignore_index=True)
            iter_page = self.__iterate_pages__()['last_page']

        current_page = self.__get_current_page__()
        print(f"Last Page Reached: {current_page}")
        print("*" * 50 + "\n")
        return df_reports

    def __get_headings__(self):
        report_headings_xpath = """//div[@class="ant-table-column-sorters-with-tooltip"]"""
        report_header = self.__webdriver.find_element_by_xpath(report_headings_xpath)
        report_headings = report_header.find_elements_by_xpath("//th")
        headings_list = []

        for heading in report_headings:
            if heading.text:
                headings_list.append(heading.text)

        return headings_list

    def __get_report_body__(self):
        headings = self.__get_headings__()
        table_body = self.__webdriver.find_element_by_xpath("""//tbody[@class="ant-table-tbody"]""")
        table_info_tds = table_body.find_elements_by_xpath("//td")

        td_text_list = []

        for td in table_info_tds:
            # if td.text != "":
            #     td_text_list.append(td.text)
            td_text_list.append(td.text)

        # Iterate through all TD and assign headings in a dictionary via reset loops
        entries_dict_list = []
        entries_dict = {}
        temp_dict = {}
        headings_counter = 0
        for index, string in enumerate(td_text_list):
            print(f"{headings_counter}: {headings[headings_counter]}: {index}: {string}")
            temp_dict.update({headings[headings_counter]: string})
            entries_dict.update(temp_dict)
            headings_counter = headings_counter + 1
            temp_dict = {}
            if headings_counter == len(headings):
                headings_counter = 0
                entries_dict_list.append(entries_dict)
                entries_dict = {}

        return pd.DataFrame(entries_dict_list)

    def __iterate_pages__(self):
        current_page = self.__get_current_page__()
        print(f"Reading Page: {current_page}")
        print("*" * 50 + "\n")

        next_page = self.__webdriver.find_element_by_xpath("""//li[contains(@class, "ant-pagination-next")]""")
        next_disabled = (next_page.get_attribute("aria-disabled"))
        if next_disabled == "false":
            next_page.click()

            return {"last_page": "false"}
        else:
            return {"last_page": "true"}

    def __get_current_page__(self):
        page_list_items = self.__webdriver.find_elements_by_xpath("""//li[contains(@class,"ant-pagination-item ant-pagination-item")]""")

        for page in page_list_items:
            page_class = page.get_attribute("class")

            if re.search(r"active", page_class):
                return page.text


In [ ]:
reports_obj = Get_Reports(c_driver)

In [ ]:
df_active_reports = reports_obj.get_reports_dataframe()

In [ ]:
df_active_reports

In [ ]:
df_active_reports.drop_duplicates(inplace=True)

In [ ]:
df_active_reports = df_active_reports[df_active_reports['Name'] != ""]

In [ ]:
df_active_reports.to_csv("[REDACTED]/.csv", index=False)

### Import Modules, Create env Variables

In [ ]:
from ts_api_connections import datto_rmm_api

## Create DataFrame via API Call Iteration


In [ ]:
dattormm_devices = datto_rmm_api.create_datto_dataframe(env_file="d:/git/config/env.ini")

In [ ]:
list_of_clients = list(dattormm_devices['siteName'].unique())

In [ ]:
report_clients_list = []


def missing_software_distribution_report(dataframe=df_active_reports, client_list=list_of_clients):
    report_found_list = []
    for name in dataframe['Name']:
        try:
            split_name = re.search('Software\sDistribution\s\-\s([^\n]+)', name).group(1)
            if split_name:
                report_found_list.append(split_name)
            else:
                print(f"Unable to split{name}")
        except Exception as e:
            pass

    report_missing_list = []
    for client in client_list:

        result = re.match(r'example', client)

        if (client not in report_found_list) & (result is None):
            report_name = f"*Monthly Reports - Software Distribution - {client}"
            report_missing_list.append(report_name)
            print(report_name)

    return report_missing_list


def missing_remote_report(dataframe=df_active_reports, client_list=list_of_clients):
    report_found_list = []
    for name in dataframe['Name']:
        try:
            split_name = re.search('Remote\sActivity\s\-\s([^\n]+)', name).group(1)
            if split_name:
                report_found_list.append(split_name)
            else:
                print(f"Unable to split{name}")
        except Exception as e:
            pass

    report_missing_list = []
    for client in client_list:

        result = re.match(r'example', client)

        if (client not in report_found_list) & (result is None):
            report_name = f"*Monthly Reports - Remote Activity - {client}"
            report_missing_list.append(report_name)
            print(report_name)

    return report_missing_list

In [ ]:
missing_remote_activity = missing_remote_report()
df_missing_reports = pd.DataFrame(missing_remote_activity)
df_missing_reports.to_csv("[REDACTED]/.csv", index=False)

In [ ]:
missing_software_distribution = missing_software_distribution_report()
df_missing_reports = pd.DataFrame(missing_software_distribution)
df_missing_reports.to_csv("[REDACTED]/.csv", index=False)

In [ ]:
missing_software_distribution

In [ ]:
missing_remote_activity